# Load packages

In [1]:
import scipy.io
import os
import pickle

# Define functions to convert matlab data objects into pandas DataFrames

In [2]:
import pandas as pd
import numpy as np

def mat2DF(mat):

    # convert the matlab object entries to lists or numpy arrays
    [t, tVR, tFrameGrab, OffsetRot, OffsetFor, OffsetLat] = getArrayFromMatObj(mat)
    
    # get imaging parameters
    [num_planes, minFG, maxFG] = getImParams(t,tVR,tFrameGrab,mat)
    
    # Match the position data to the framegrab times
    # (the imaging experiment and the virtual reality session overlap but do not exactly coincide)
    OffsetRotMatch = np.zeros((maxFG-minFG+1,2))
    OffsetForMatch = np.zeros(maxFG-minFG+1)
    OffsetLatMatch = np.zeros(maxFG-minFG+1)
    for interp in range(minFG,maxFG):
        tMatch = np.where(t >= (t[0] + (tFrameGrab[(interp-1)*num_planes]-tVR[0])/10000))[0]
        OffsetRotMatch[interp-minFG,0] = t[tMatch[0]]
        OffsetRotMatch[interp-minFG,1] = np.pi/180*OffsetRot[tMatch[0]]
        OffsetForMatch[interp-minFG] = OffsetFor[tMatch[0]]
        OffsetLatMatch[interp-minFG] = OffsetLat[tMatch[0]]

    # Load the data in a DataFrame
    DF = pd.DataFrame({'t':OffsetRotMatch[:,0],
                       'angle':OffsetRotMatch[:,1],
                       'forward position':OffsetForMatch,
                       'lateral position':OffsetLatMatch})
    for r in range(mat['GROIaveMax'].shape[0]):
        DF['roi ' + str(r) + ' Green'] = mat['GROIaveMax'][r,minFG-1:maxFG]
    for r in range(mat['RROIaveMax'].shape[0]):
        DF['roi ' + str(r) + ' Red'] = mat['RROIaveMax'][r,minFG-1:maxFG]

    return DF

def getArrayFromMatObj(mat):
    """
    Convert MATLAB object values into lists
    
    Parameters
    ----------
    mat: a dictionary loaded in from the original MATLAB data file

    Returns
    -------
    t: the time
    tVR: indices when the virtual reality refreshed
    tFrameGrab: indices when the microscope grabbed a frame
    OffsetRot: the fly's angle
    OffsetFor: the fly's x position
    OffsetLat: the fly's y position
    """ 
    t = [a[0] for a in mat['positionDat']['t'][0][0]]
    tVR = mat['positionDat']['tVR'][0][0][0]
    tFrameGrab = mat['positionDat']['tFrameGrab'][0][0][0]  
    OffsetRot = [a[0] for a in mat['positionDat']['OffsetRot'][0][0]]
    OffsetFor = [a[0] for a in mat['positionDat']['OffsetFor'][0][0]]
    OffsetLat = [a[0] for a in mat['positionDat']['OffsetLat'][0][0]]

    return [t, tVR, tFrameGrab, OffsetRot, OffsetFor, OffsetLat]

def getImParams(t,tVR,tFrameGrab,mat):
    """
    Get parameters related to calcium imaging 
    
    Parameters
    ----------
    mat: a dictionary loaded in from the original MATLAB data file
    t: the time
    tVR: indices when the virtual reality refreshed
    tFrameGrab: indices when the microscope grabbed a frame

    Returns
    -------
    num_planes: the number of planes in the imaging volume
    minFG: the first point at which both the microscope and the virtual reality were recording data
    maxFG: the last point of simultaneous data acquisition
    """ 
    num_planes = int(len(tFrameGrab)/mat['GROIaveMax'].shape[1])
    minFG = int(np.ceil(np.min(np.where(tFrameGrab >= tVR[0]))/num_planes))
    maxFG = int(np.round(len(tFrameGrab)/num_planes))

    return [num_planes, minFG, maxFG]

# Load data into a dictionary of pandas notebooks and save

In [3]:
# Specify the parent directories
parent_dirs = ['PEN1-EB','PEN1-PB']

# Initialize a dictionary to hold all of the data
allDat = {}

# Step through the directories to the files
for d in parent_dirs:
    dates = [dt for dt in os.listdir(os.path.join('../data',d)) if not dt.startswith('.')]
    flyID = 0
    allDat[d] = {}
    for dt in dates:
        files = os.listdir(os.path.join('../data',d,dt))
        files.sort()
        flyNm = ''
        for f in files:   
            # Load the data for a given trial
            mat = scipy.io.loadmat(os.path.join('../data',d,dt,f))
            # Convert it to a pandas DataFrame
            DF = mat2DF(mat)

            # Pull out values specific to the experiment
            nm = f.split('_')[0]
            if nm != flyNm:
                flyNm = nm
                flyID += 1
                fly = 'Fly ' + str(flyID)
                allDat[d][fly] = {}
                allDat[d][fly]['date'] = dt
                allDat[d][fly]['age'] = f.split('_')[1]
                allDat[d][fly]['colors'] = f.split('_')[2:4]
                allDat[d][fly]['Dark'] = {}
                allDat[d][fly]['1x'] = {}
                allDat[d][fly]['2x'] = {}
       
            trial = 'Trial ' + str(f[-5])
            gain = f.split('_')[-2]
            allDat[d][fly][gain][trial] = DF

In [12]:
# Pickle the data and save it
output = open('../data/PEN1data.pkl', 'wb')
pickle.dump(allDat, output)
output.close()